In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# Embed

## Get Input Tensor

In [2]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
# l

recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

data/ProcData/TensorFolder/Task2YearXXX


In [3]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX/P@age-AgeNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX/P@basicInfo-basicInfoDftGrn
data/ProcData/TensorFolder/Task2YearXXX/EC@BasicInfo-BasicDftGrn
data/ProcData/TensorFolder/Task2YearXXX/EC@DT_min-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/A1C@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX/Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX/PNSectSent@Sentence-Tk@TknzLLMGrn
B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [4]:
full_recfldgrn_list = [k for k in batch_rfg]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [5]:
import torch 

RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

## Get df_SubUnit

### Get Original df_SubUnit

In [6]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

############################### NEW THING: replace E with EL
default_SubUnitName = 'EL'
fldgrn_folder = 'data/ProcData/FldGrnInfo'


import pandas as pd
def get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_R_subunit_name = 'E'):
    SubUnit_List = []
    for input_name in full_recfldgrn_list:
        d = {}
        d['SubUnitName'] = default_R_subunit_name
        
        recfldgrn = '-'.join(input_name.split('-')[-2:])
        output_layerid = len(input_name.split('-'))
        output_name = input_name.split('Grn')[0]
        
        d['input_names'] = [input_name]
        d['output_name'] = output_name
        
        d['output_layerid'] = output_layerid
        
        SubUnit_List.append(d)
        
    df_SubUnit = pd.DataFrame(SubUnit_List)
    
    return df_SubUnit


default_R_subunit_name = 'EL'
df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_R_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid
0,EL,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3
1,EL,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3
2,EL,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4
3,EL,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4
4,EL,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5
5,EL,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5
6,EL,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5
7,EL,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5
8,EL,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7


### Get SubUnit Name's NN List

In [7]:
def mapping_SubUnitName_to_SubUnitNNList(SubUnitName, input_names, 
                                         default_BasicNNtype_To_NNName):
    SubUnitNNList = []
    for idx, NAME in enumerate(SubUnitName):
        if NAME == 'E':
            nn_type = 'expander'
            input_name = input_names[0]
            
            assert 'Grn' in input_name
            assert len(input_names) == 1
            
            if 'Tknz' in input_name:
                nn_name = 'LLMEmbed'
            elif '_wgt' in input_name:
                nn_name = 'NumeEmbed'
            else:
                nn_name = 'CateEmbed'
            
        elif NAME == 'R':
            if idx == 0: assert len(input_names) == 1
            nn_type = 'reducer'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'M':
            nn_type = 'merger'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'L':
            assert len(input_names) == 1
            nn_type = 'learner'
            input_name = input_names[0]
            layers_num = len(input_name.split('-'))
            if layers_num >= 3:
                nn_name = 'TFM'
            elif layers_num == 2:
                nn_name = 'Linear'
            else:
                raise ValueError(f'incorrect layers_num {layers_num}')
            
        else:
            raise ValueError(f'The BasicNN is not correct {NAME}')
        
        SubUnitNNList.append(nn_type + '-' + nn_name)
    return SubUnitNNList

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s

df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List
0,EL,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,"[expander-NumeEmbed, learner-TFM]"
1,EL,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,"[expander-CateEmbed, learner-TFM]"
2,EL,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,"[expander-CateEmbed, learner-TFM]"
3,EL,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,"[expander-CateEmbed, learner-TFM]"
4,EL,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,"[expander-CateEmbed, learner-TFM]"
5,EL,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,"[expander-NumeEmbed, learner-TFM]"
6,EL,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,"[expander-CateEmbed, learner-TFM]"
7,EL,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,"[expander-CateEmbed, learner-TFM]"
8,EL,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,"[expander-LLMEmbed, learner-TFM]"


### Get Default BasicNN Config List

In [8]:
from fieldnn.dataflowfn.embedflowfn import get_Default_ExpanderNNPara

def get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, 
                                    fldgrn_folder, learner_default_dict):
    
    SubUnit_DefaultBasicNN_List = []
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):
        nn_type, nn_name = nn_type_nn_name.split('-')
        
        if basic_nn_idx == 0:
            # the first BasicNN in the SubUnit
            # expander and merger will only be here.
            # this assigments only works for the first iteration
            # let's make a village rule: only E, R, M can be the first. 
            assert nn_type in ['expander', 'reducer', 'merger']
        
            if nn_type == 'expander':
                input_names_nnlvl = SubUnit_input_names 
                assert len(input_names_nnlvl) == 1
                full_recfldgrn = input_names_nnlvl[0]
                default_para = get_Default_ExpanderNNPara(full_recfldgrn, fldgrn_folder)
                
            else:
                default_para = {}
            
        else:
            assert nn_type in ['reducer', 'learner']
            if nn_type == 'learner':
                default_para = learner_default_dict[nn_name] # TODO
            else:
                default_para = {}
        
        SubUnit_DefaultBasicNN_List.append(default_para)    
        
    return SubUnit_DefaultBasicNN_List
    


###########################################################
psn_embedprocess = {
    # 'activator': 'gelu',
    # 'dropout': {'p': 0.5, 'inplace': False},
    # 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}
}

learner_default_dict = {
    'TFM': {'psn_max': 512, 
            'psn_embedprocess': psn_embedprocess},
    'Linear': {'initrange': 0.1},   
}
###########################################################
    
    
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s

df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,EL,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,"[expander-NumeEmbed, learner-TFM]","[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ..."
1,EL,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, 'Male': 1..."
2,EL,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, 'X': 1, '..."
3,EL,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, '_missing..."
4,EL,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, '_missing..."
5,EL,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,"[expander-NumeEmbed, learner-TFM]","[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ..."
6,EL,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, '_missing..."
7,EL,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, '_missing..."
8,EL,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,"[expander-LLMEmbed, learner-TFM]",[{'vocab_tokenizer': BertTokenizerFast(name_or...


### Get BasicNN Config List

In [9]:
from fieldnn.configfn.expanderfn import get_expander_para
from fieldnn.configfn.mergerfn import get_merger_para
from fieldnn.configfn.reducerfn import get_reducer_para
from fieldnn.configfn.learnerfn import get_learner_para

def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    if nn_type == 'expander':
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].split('Grn')[0]
        
        # Get the input_size and output_size
        input_size = None
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process
        
        # Derive the para
        vocab_tokenizer = default_nnpara['vocab_tokenizer']
        init = default_nnpara['init']
        para = get_expander_para(nn_name, default_nnpara, 
                                 embed_size, vocab_tokenizer, init, postprocess)

    elif nn_type == 'reducer': 
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '@' + fld)

        # Get the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size

        # Get the output_size
        if nn_name.lower() == 'concat':
            # this types of merger only happened in the R in MLRL. 
            # you can skip this part safely if you haven't encountered M. 
            fld_childflds = input_names_nnlvl[0].split('-')[-1]
            assert '@' in fld_childflds
            childflds = fld_childflds.split('@')[-1]
            childflds = childflds.split('&')
            output_size = len(childflds) * embed_size
        else:
            # usually the most case. 
            output_size = embed_size 

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, default_nnpara, input_size, output_size, postprocess)  
        
    elif nn_type == 'merger':
        # generate the output name
        
        assert len(input_names_nnlvl) > 1
        # input_names_nnlvl: ['B-P-EC-A1C@DT', 'B-P-EC-A1C@V']
        
        childflds = [i.split('-')[-1] for i in input_names_nnlvl]
        # childflds: ['A1C@DT', 'A1C@V']
        
        fld_childflds = '&'.join([i.split('@')[-1] for i in childflds])
        # fld_childflds: DT&V
        
        output_prefix = input_names_nnlvl[0].replace('@' + childflds[0].split('@')[-1], '')
        # output_prefix: B-P-EC-A1C
        
        output_name_nnlvl = output_prefix + '-' + fld_childflds
        # output_name_nnlvl: B-P-EC-A1C-DT&V
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_merger_para(nn_name, default_nnpara, input_size, output_size, postprocess) 
        
        
    elif nn_type == 'learner':
        # generate the output name
        assert len(input_names_nnlvl) == 1
        output_name_nnlvl = input_names_nnlvl[0] # just the same name as before
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_learner_para(nn_name, default_nnpara, input_size, output_size, postprocess)
        
        
    else:
        raise ValueError(f'nn_type {nn_type} is not available yet')

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                    'output_name_nnlvl': output_name_nnlvl, 
                    f'{nn_type}_para': para}
    
    return Basic_Config


def get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   ):
    
    
    # TODO: also add the layer_idx in order to deal with the learn_layer_para.
    # This function also needs to be updated. 
    
    SubUnit_BasicNN_Config_List = []
    
    # print('\n\n************** SubUnit_BasicNN_List ****************')
    # print(SubUnit_BasicNN_List)
    # print(SubUnit_input_names)
    # print(SubUnit_output_name)
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):

        # Get the input_names_nnlvl
        if basic_nn_idx == 0:
            input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
        else:
            input_names_nnlvl = [output_name_nnlvl]
        
        ##############
        default_nnpara = SubUnit_DefaultBasicNN_List[basic_nn_idx] 
        ##############
        
        
        Basic_Config = generate_BasicNN_Config(nn_type_nn_name, 
                                               input_names_nnlvl, 
                                               default_nnpara, 
                                               embed_size, 
                                               process)
        output_name_nnlvl = Basic_Config['output_name_nnlvl']
        BasicNN_Config = {'nn_type_nn_name': nn_type_nn_name, 'Basic_Config': Basic_Config}
        SubUnit_BasicNN_Config_List.append(BasicNN_Config)
        
        # print('==========================')
        # print(basic_nn_idx, nn_type_nn_name)
        # print(input_names_nnlvl, '<-------- input_names_nnlvl')
        # print(output_name_nnlvl, '<-------- output_name_nnlvl')
        
        
        # also check the input_size of dim and output_size of dim

    final_output_name_nnlvl = output_name_nnlvl

    # if not SubUnit_output_name in final_output_name_nnlvl:
    #     print('xxx errors xxx')
    #     print(final_output_name_nnlvl, '<------- final_output_name_nnlvl')
    #     print(SubUnit_output_name, '<------- SubUnit_output_name')
    #     print('xxx errors xxx')
    assert SubUnit_output_name in final_output_name_nnlvl
    # print('\n************** End SubUnit_BasicNN_List ****************')
    return SubUnit_BasicNN_Config_List




In [10]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,EL,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,"[expander-NumeEmbed, learner-TFM]","[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,EL,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, 'Male': 1...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,EL,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, 'X': 1, '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
3,EL,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
4,EL,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
5,EL,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,"[expander-NumeEmbed, learner-TFM]","[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
6,EL,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
7,EL,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,"[expander-CateEmbed, learner-TFM]","[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
8,EL,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,"[expander-LLMEmbed, learner-TFM]",[{'vocab_tokenizer': BertTokenizerFast(name_or...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


## Embed Block

In [11]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


# Repr

## Get Input Tensor

In [12]:
for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


## Get df_SubUnit
### Get Original df_SubUnit 

In [13]:
from fieldnn.dataflowfn.reprflowfn import get_Repr_dataflow_table
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow_completename
from fieldnn.dataflowfn.reprflowfn import get_Repr_SubUnit_List

In [14]:
full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [15]:
df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
# df_dataflow

df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'ReducerOnly')
# df_dataflow_new

df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
# df_dataflow_new

In [16]:

def get_Repr_SubUnit_List(df_dataflow, 
                          default_R_subunit_name = 'RL', 
                          default_MR_subunit_name = 'MRL', # or 'MLRL'
                         ):
    layeridx_list = list(df_dataflow.columns)
    
    SubUnit_List = []
    for idx in range(len(layeridx_list) - 1):
        A_layerid = layeridx_list[idx]
        B_layerid = layeridx_list[idx + 1]
        # print(A_layerid, B_layerid)

        A_tensors = df_dataflow[A_layerid]
        A_tensors = A_tensors[-A_tensors.isna()].to_dict()

        B_tensors = df_dataflow[B_layerid]
        B_tensors = B_tensors[-B_tensors.isna()].to_dict()

        # print(f'\nFrom Layer {A_layerid} to {B_layerid}:')

        # from A tensor to B tensor, we have the Reducer NNs.
        # also notice that some 

        # print('A', A_tensors)
        for k in A_tensors:
            if k not in B_tensors: continue

            input_name = A_tensors[k]
            output_name = B_tensors[k]

            if pd.isna(output_name) == True: continue # pass it. 

            d = {}
            d['SubUnitName'] = default_R_subunit_name
            d['input_names'] = [input_name]
            d['output_name'] = output_name
            
            d['input_layerid'] = A_layerid
            d['output_layerid'] = B_layerid
        
        
            SubUnit_List.append(d)


        # from B tensor: potential there are some Merger NNs. 
        # print('B', B_tensors)
        # check whether these is a '(Merger') in the key
        merger_tensors = [v for k, v in B_tensors.items() if '(Merge)' in k and '@' not in v]
        # print('B-merger_tensors', merger_tensors)

        for output_tensor in merger_tensors:
            input_tensors = [i for k, i in B_tensors.items() if output_tensor + '@' in i]
            # print(output_tensor, ':', input_tensors)

            d = {}
            d['SubUnitName'] = default_MR_subunit_name
            d['input_names'] = input_tensors
            d['output_name'] = output_tensor
            
            d['input_layerid'] = B_layerid
            d['output_layerid'] = B_layerid
        
            SubUnit_List.append(d)
            
    df_SubUnit = pd.DataFrame(SubUnit_List)
    return df_SubUnit



In [17]:
df_dataflow = df_dataflow_new.copy()
default_R_subunit_name = 'RL' # 'RL'
default_MR_subunit_name = 'MRL' # 'MLRL', 'MRL', 'MR'
df_SubUnit = get_Repr_SubUnit_List(df_dataflow, default_R_subunit_name, default_MR_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,RL,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,RL,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,RL,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,RL,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,RL,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,RL,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,RL,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,MRL,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4
8,MRL,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4
9,RL,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3


### get SubUnit Name's NN List


Example
* E --> [expander-LLM]
* MR --> [merger-Merge, reducer-ReduceMax]


In [18]:
# from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList

def mapping_SubUnitName_to_SubUnitNNList(SubUnitName, input_names, 
                                         default_BasicNNtype_To_NNName):
    SubUnitNNList = []
    for idx, NAME in enumerate(SubUnitName):
        if NAME == 'E':
            nn_type = 'expander'
            input_name = input_names[0]
            
            assert 'Grn' in input_name
            assert len(input_names) == 1
            
            if 'Tknz' in input_name:
                nn_name = 'LLMEmbed'
            elif '_wgt' in input_name:
                nn_name = 'NumeEmbed'
            else:
                nn_name = 'CateEmbed'
            
        elif NAME == 'R':
            if idx == 0: assert len(input_names) == 1
            nn_type = 'reducer'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'M':
            nn_type = 'merger'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'L':
            # print(input_names)
            assert idx != 0
            # assert len(input_names) == 1; not necessary, L can follow M
            nn_type = 'learner'
            input_name = input_names[0]
            layers_num = len(input_name.split('-'))
            if layers_num >= 3:
                nn_name = 'TFM'
            elif layers_num == 2:
                nn_name = 'Linear'
            else:
                raise ValueError(f'incorrect layers_num {layers_num}')
            
        else:
            raise ValueError(f'The BasicNN is not correct {NAME}')
        
        SubUnitNNList.append(nn_type + '-' + nn_name)
    return SubUnitNNList

In [19]:
############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)


df_SubUnit['SubUnit_BasicNN_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List
0,RL,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,"[reducer-Max, learner-TFM]"
1,RL,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,"[reducer-Max, learner-TFM]"
2,RL,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,"[reducer-Max, learner-TFM]"
3,RL,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,"[reducer-Max, learner-TFM]"
4,RL,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,"[reducer-Max, learner-TFM]"
5,RL,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,"[reducer-Max, learner-TFM]"
6,RL,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,"[reducer-Max, learner-TFM]"
7,MRL,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max, learner-TFM]"
8,MRL,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max, learner-TFM]"
9,RL,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,"[reducer-Max, learner-TFM]"


### Get Default BasicNN Config List

In [20]:

    

from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List

###########################################################
psn_embedprocess = {
    # 'activator': 'gelu',
    # 'dropout': {'p': 0.5, 'inplace': False},
    # 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}
}

learner_default_dict = {
    'TFM': {'psn_max': 512, 
            'psn_embedprocess': psn_embedprocess},
    'Linear': {'initrange': 0.1},   
}
###########################################################
    
    
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s

df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,RL,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
1,RL,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
2,RL,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
3,RL,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
4,RL,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
5,RL,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
6,RL,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
7,MRL,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max, learner-TFM]","[{}, {}, {'psn_max': 512, 'psn_embedprocess': ..."
8,MRL,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max, learner-TFM]","[{}, {}, {'psn_max': 512, 'psn_embedprocess': ..."
9,RL,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"


### Get BasicNN Config List

In [21]:
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

In [22]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,RL,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
1,RL,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
2,RL,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
3,RL,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
4,RL,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
5,RL,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
6,RL,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
7,MRL,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max, learner-TFM]","[{}, {}, {'psn_max': 512, 'psn_embedprocess': ...","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
8,MRL,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max, learner-TFM]","[{}, {}, {'psn_max': 512, 'psn_embedprocess': ...","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
9,RL,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."


## Repr Block

In [23]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [24]:
fld_updates_dict = {}
for i in RECFLD_TO_EMBEDTESNOR:
    layernum = len(i.split('-'))
    fld = i.split('-')[-1]
    if '@' in fld:
        # print(fld)
        neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
        # print(neat_i)
        same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
        # print(same_neat_list)
        if len(same_neat_list) == 1: # itself
            # RECFLD_TO_EMBEDTESNOR[]
            fld_updates_dict[i] = neat_i
            
for old, new in fld_updates_dict.items():
    RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)
    
    
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [25]:
# from fieldnn.module.reprblock import ReprBlockLayer

import torch
from fieldnn.module.subunit import SubUnit_Layer


class ReprBlockLayer(torch.nn.Module):
    
    def __init__(self, df_SubUnit):
        super(ReprBlockLayer, self).__init__()
        self.df_SubUnit = df_SubUnit
        self.SubUnitDict = torch.nn.ModuleDict()
        
        for idx, SubUnit_info in df_SubUnit.iterrows():
            output_name = SubUnit_info['output_name']
            SubUnitLayer = SubUnit_Layer(SubUnit_info)
            self.SubUnitDict[output_name] = SubUnitLayer

    def forward(self, RECFLD_TO_EMBEDTESNOR):
        
        OUTPUT_TO_TENSOR = RECFLD_TO_EMBEDTESNOR.copy()
        
        for output_name, SubUnitLayer in self.SubUnitDict.items():
            input_names = SubUnitLayer.SubUnit_input_names
            # print(input_names)
            # for i in input_names:
            #     print(i, OUTPUT_TO_TENSOR[i]['info'].shape)
            SubUnit_output_name, info_dict = SubUnitLayer(input_names, OUTPUT_TO_TENSOR)
            
            assert output_name == SubUnit_output_name
            OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict
        
        return OUTPUT_TO_TENSOR
    

In [26]:


ReprBlock = ReprBlockLayer(df_SubUnit)

OUTPUT_TO_TESNOR = ReprBlock(RECFLD_TO_EMBEDTESNOR)

for k, v in OUTPUT_TO_TESNOR.items():
    print(k, v['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk torch.Size([4, 25, 1, 14, 121, 11, 128])
B-P-EC-PN-PNSect-PNSectSent torch.Size([4, 25, 1, 14, 121, 128])
B-P-EC-PN-PNSect torch.Size([4, 25, 1, 14, 128])
B-P-EC-A1C@DT torch.Size([4, 25, 1, 128])
B-P-EC-A1C@V torch.Size([4, 25, 1, 128])
B-P-EC-Diag@DT torch.Size([4, 25, 22, 128])
B-P-EC-Diag@Value torch.Size([4, 25, 22, 128])
B-P-EC-PN torch.Size([4, 25, 1, 128])
B-P-EC-A1C torch.Size([4, 25, 1, 128])
B-P-EC-Diag torch.Size([4, 25, 22, 128])
B-P-EC@BasicInfo torch.Size([4, 25, 128])
B-P-EC@DT_min torch.Size([4, 25, 128])
B-P-EC@A1C torch.